Basic doc:                                          https://docs.llamaindex.ai/en/stable/examples/low_level/oss_ingestion_retrieval/

Quickstart on working with chroma collections:      https://www.analyticsvidhya.com/blog/2023/07/guide-to-chroma-db-a-vector-store-for-your-generative-ai-llms/

In [ ]:
# import pandas as pd

# data = pd.read_csv("../../dataset_full_texts.csv")

In [ ]:
# data.columns

In [ ]:
# # grabbing some examples to add to the vector store

# text_examples = data["extracted_text"].sample(5)
# text_examples.to_csv("rag_examples.csv", index=False)

# Setting up the vector database

In [1]:
# requires
!pip install Cython llama-index-llms-ollama llama-index-embeddings-huggingface llama-index-llms-llama-cpp chroma llama-index-embeddings-openai llama_index-vector-stores-chroma llama-index-llms-openai

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.7/615.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━

store the text or the pdfs? we can use multimodal models

In [4]:
import pandas as pd

text_examples = pd.read_csv("rag_examples.csv")

In [ ]:
text_examples.columns

Index(['extracted_text'], dtype='object')

In [5]:
text_examples_list = text_examples["extracted_text"].to_list()

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb

# we can use PINECODE for cloud storage but idk if their free tier would be sufficient

In [7]:
chroma_client = chromadb.PersistentClient(path="rag_examples") 
chroma_collection = chroma_client.get_or_create_collection("collection_example")

In [8]:
from llama_index.core import Settings

embed_model = HuggingFaceEmbedding(model_name="dangvantuan/sentence-camembert-base", embed_batch_size=1) # subject to change, big batch size will make it crash
Settings.embed_model = embed_model # step needed, the index store will call OpenAI embeddings by default

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.27k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

Saving Docs to chromaDB

In [10]:
cut_down_list_texts = [t[0:356] for t in text_examples_list[0:2]]

In [11]:
from llama_index.core import Document

documents = [Document(text=cut_down_list_texts[0])]
# documents = [Document(text=t) for t in cut_down_list_texts]

In [ ]:
# from llama_index.core.node_parser import SentenceSplitter

# parser = SentenceSplitter()
# nodes = parser.get_nodes_from_documents(documents)

# the VecoreStoreIndex object accepts data type NODE or DOCUMENT

# so we have to manually convert strings if we havent used their data_loaders
# (we could use their native data_loaders if we have the pdfs)

: 

In [12]:
index = VectorStoreIndex.from_documents(documents,
                                        storage_context=storage_context,
                                        embed_mode=embed_model) # the embeddings for the docs are computed in this step

In [15]:
chroma_collection.get()
# calling the vectorstoreindex doesnt save the embeddings to the db automatically

{'ids': ['d1a283e7-1ad8-438f-ae08-242b62d75777'],
 'embeddings': None,
 'documents': ['Mis en ligne le : 12/07/2022 - vUle t1eV n1ers.-le.-bel Séance du Conseil Municipal du vendredi l juillet 2022 Extrait du Registre des Délibérations du Conseil Municipal Ville de Villiers-le-bel Séance ordinaire du vendredi 1 juillet 2022 N°1/Com pte rendu Compte rendu commenté (procès-verbal) de la séance du Conseil Municipal du 24 mai 2022 Le vendredi'],
 'uris': None,
 'data': None,
 'metadatas': [{'_node_content': '{"id_": "d1a283e7-1ad8-438f-ae08-242b62d75777", "embedding": null, "metadata": {}, "excluded_embed_metadata_keys": [], "excluded_llm_metadata_keys": [], "relationships": {"1": {"node_id": "d6378662-4221-4092-a800-707398c4e601", "node_type": "4", "metadata": {}, "hash": "bbda94ad413e22d8f56b9d2abd5d73a246260a9642c532fe2127c7dec1f28f5e", "class_name": "RelatedNodeInfo"}}, "text": "", "mimetype": "text/plain", "start_char_idx": 0, "end_char_idx": 355, "text_template": "{metadata_str}\\n\\

In [16]:
index.storage_context.persist("rag_examples/camembert")

To load the Embeddings and create the query engine we  will have to pass **persist_dir="rag_examples/camembert"** to the StorageContext object when we load it.

In [19]:
!zip -r rag_examples.zip rag_examples

  adding: rag_examples/ (stored 0%)
  adding: rag_examples/camembert/ (stored 0%)
  adding: rag_examples/camembert/docstore.json (deflated 21%)
  adding: rag_examples/camembert/graph_store.json (stored 0%)
  adding: rag_examples/camembert/image__vector_store.json (deflated 19%)
  adding: rag_examples/camembert/index_store.json (deflated 40%)
  adding: rag_examples/c5306bb5-e9bc-45cd-a2f0-d1b8628642d4/ (stored 0%)
  adding: rag_examples/c5306bb5-e9bc-45cd-a2f0-d1b8628642d4/length.bin (deflated 7%)
  adding: rag_examples/c5306bb5-e9bc-45cd-a2f0-d1b8628642d4/link_lists.bin (stored 0%)
  adding: rag_examples/c5306bb5-e9bc-45cd-a2f0-d1b8628642d4/data_level0.bin (deflated 40%)
  adding: rag_examples/c5306bb5-e9bc-45cd-a2f0-d1b8628642d4/header.bin (deflated 61%)
  adding: rag_examples/chroma.sqlite3 (deflated 93%)


In [20]:
from google.colab import files
files.download("rag_examples.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>